In [ ]:
import geemap
import ee

ee.Authenticate()
# Replace 'your-project-id' with your actual Google Cloud Project ID
ee.Initialize(project='california-fire-model')

Map = geemap.Map()
california = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq('NAME', 'California'))

Use percentile-based thresholds for better balance
This does not misrepresent reality — it re-scales the labels to allow balanced learning and visualization.

This isn’t “lying with data” — it’s binning based on relative fire risk, which helps CNN learn general patterns, not absolute counts.


 Fire frequency (burned years) across California:
Most areas: 0–2 fires in 21 years → makes up ~80%+

Some hotspots: 15–20 fires in 21 years → limited spatially

Very few areas: fall in that 5–10 fire count range (~24%–48% recurrence)

So your medium bin (5–10 years) captures a very narrow group, and statistically, it's valid that it's nearly empty — but it’s not helpful for visualization or model training.

In [ ]:
from google.colab import files
import os

# Download all maps from the folder
for season in ['DJF', 'MAM', 'JJA', 'SON']:
    filepath = f'evm_maps/{season}_tasmax_map.png'
    files.download(filepath)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

# Step 1: Install and import packages
!pip install earthengine-api geemap --quiet

import ee
import geemap

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='california-fire-model')

# Step 2: Define California geometry
california = ee.FeatureCollection("TIGER/2018/States") \
    .filter(ee.Filter.eq("NAME", "California")) \
    .geometry()

# Step 3: Load MODIS burned area dataset
modis_ba = ee.ImageCollection("MODIS/061/MCD64A1").select('BurnDate')

# Step 4: Define seasons
seasons = {
    'DJF': ([12, 1, 2], 'Dec-Feb'),
    'MAM': ([3, 4, 5], 'Mar-May'),
    'JJA': ([6, 7, 8], 'Jun-Aug'),
    'SON': ([9, 10, 11], 'Sep-Nov')
}

def get_season_dates(year, months):
    start_month = months[0]
    end_month = months[-1]
    if start_month > end_month:
        start = ee.Date(f'{year-1}-{start_month:02d}-01')
        end = ee.Date(f'{year}-{end_month:02d}-28')
    else:
        start = ee.Date(f'{year}-{start_month:02d}-01')
        end = ee.Date(f'{year}-{end_month:02d}-28')
    return start, end

# Step 5: Generate seasonal risk maps
seasonal_risk_maps = {}

for season_key, (months, label) in seasons.items():
    print(f"Processing multi-class BIM for: {season_key} ({label})")

    seasonal_bims = []

    for year in range(2005, 2026):
        start, end = get_season_dates(year, months)
        seasonal_fire = modis_ba.filterDate(start, end).map(lambda img: img.gt(0).rename('Fire'))

        count = seasonal_fire.size()
        season_sum = ee.Image(ee.Algorithms.If(
            count.gt(0),
            seasonal_fire.sum().clip(california),
            ee.Image.constant(0).rename('Fire').clip(california)
        ))

        bim = season_sum.gt(0).rename('BIM')
        seasonal_bims.append(bim)

    avg_bim = ee.ImageCollection(seasonal_bims).mean().clip(california).reproject(crs='EPSG:4326', scale=500)
    avg_bim = avg_bim.rename('BIM')

    stats = avg_bim.reduceRegion(
        reducer=ee.Reducer.percentile([33, 66]),
        geometry=california,
        scale=500,
        maxPixels=1e13
    )

    p33 = ee.Number(ee.Algorithms.If(stats.contains('BIM'), stats.get('BIM'), 0.33))
    p66 = ee.Number(ee.Algorithms.If(stats.contains('BIM'), stats.get('BIM'), 0.66))

    low = avg_bim.lt(p33)
    med = avg_bim.gte(p33).And(avg_bim.lte(p66))
    high = avg_bim.gt(p66)

    risk_map = (low.multiply(0)
                .add(med.multiply(1))
                .add(high.multiply(2))) \
                .rename(f'Risk_{season_key}') \
                .unmask(0) \
                .clip(california)

    seasonal_risk_maps[season_key] = risk_map

# Step 6: Display maps
Map = geemap.Map(center=[37.5, -119], zoom=6)

risk_vis = {
    'min': 0,
    'max': 2,
    'palette': ['green', 'orange', 'red']
}

for season_key, risk_map in seasonal_risk_maps.items():
    try:
        Map.addLayer(risk_map.reproject(crs='EPSG:4326', scale=500), risk_vis, f'{season_key} Risk Map')
    except Exception as e:
        print(f"❌ Error adding layer for {season_key}: {e}")

Map.addLayer(california, {}, "California Boundary")
Map.addLayerControl()
Map



Processing multi-class BIM for: DJF (Dec-Feb)
Processing multi-class BIM for: MAM (Mar-May)
Processing multi-class BIM for: JJA (Jun-Aug)
Processing multi-class BIM for: SON (Sep-Nov)


Map(center=[37.5, -119], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

In [ ]:
# Zip all PNG files
import shutil
shutil.make_archive('Risk_Maps', 'zip', 'risk_maps_png')
from google.colab import files
files.download('Risk_Maps.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Step 1: Install and import packages
!pip install earthengine-api geemap --quiet

import ee
import geemap

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='california-fire-model')

# Step 2: Define California geometry
california = ee.FeatureCollection("TIGER/2018/States") \
              .filter(ee.Filter.eq("NAME", "California")) \
              .geometry()

# Step 3: Define seasons
seasons = {
    'DJF': ([12, 1, 2], 'Dec-Feb'),
    'MAM': ([3, 4, 5], 'Mar-May'),
    'JJA': ([6, 7, 8], 'Jun-Aug'),
    'SON': ([9, 10, 11], 'Sep-Nov')
}

def get_season_dates(year, months):
    start_month = months[0]
    end_month = months[-1]
    if start_month > end_month:
        start = ee.Date(f'{year-1}-{start_month:02d}-01')
        end = ee.Date(f'{year}-{end_month:02d}-28')
    else:
        start = ee.Date(f'{year}-{start_month:02d}-01')
        end = ee.Date(f'{year}-{end_month:02d}-28')
    return start, end

# Function to upscale to 500m resolution
def upscale(img):
    return img.resample('bilinear').reproject(crs='EPSG:4326', scale=500)

# Step 4: Loop through each season and compute EVMs
evms = {}

for season_key, (months, label) in seasons.items():
    print(f"Processing: {season_key} ({label})")

    start, end = get_season_dates(2005, months)

    # === PRISM: tmax, tmin, precip ===
    prism_tmax = ee.ImageCollection('OREGONSTATE/PRISM/AN81m') \
        .filterDate(start, end).select('tmax').mean().rename('tmax')
    prism_tmin = ee.ImageCollection('OREGONSTATE/PRISM/AN81m') \
        .filterDate(start, end).select('tmin').mean().rename('tmin')
    prism_precip = ee.ImageCollection('OREGONSTATE/PRISM/AN81m') \
        .filterDate(start, end).select('ppt').sum().rename('precip')

    # === GRIDMET: wind, humidity, eto, aridity ===
    gridmet = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET').filterDate(start, end)
    wind = upscale(gridmet.select('vs').mean().rename('wind'))
    humid = upscale(gridmet.select('sph').mean().rename('humidity'))
    eto = upscale(gridmet.select('eto').sum().rename('eto'))
    aridity = upscale(gridmet.select('pr').sum().divide(gridmet.select('eto').sum()).rename('aridity'))

    # === MODIS: NDVI, LAI ===
    ndvi = upscale(ee.ImageCollection('MODIS/006/MOD13A1') \
        .filterDate(start, end).select('NDVI').mean().rename('NDVI'))
    lai = upscale(ee.ImageCollection('MODIS/006/MCD15A3H') \
        .filterDate(start, end).select('Lai').mean().rename('LAI'))

    # === Upscale PRISM (no unmasking for clean stats) ===
    tmax = upscale(prism_tmax)
    tmin = upscale(prism_tmin)
    precip = upscale(prism_precip)

    # === Combine all variables ===
    evm = ee.Image.cat([
        tmax, tmin, precip, wind, humid, eto,
        aridity, ndvi, lai
    ]).clip(california)

    evms[season_key] = evm

# === Step 5: Display maps using geemap ===
Map = geemap.Map(center=[37.5, -119], zoom=6)

# Season-specific tmax ranges (from previous analysis)
season_vis_ranges = {
    'DJF': (0, 20),
    'MAM': (5, 30),
    'JJA': (15, 45),
    'SON': (5, 30),
}

for season_key in ['DJF', 'MAM', 'JJA', 'SON']:
    vmin, vmax = season_vis_ranges[season_key]
    Map.addLayer(evms[season_key], {
        'bands': ['tmax'],
        'min': vmin,
        'max': vmax,
        'palette': ['blue', 'green', 'yellow', 'red']
    }, f'{season_key} PRISM Tmax (°C)')

Map.addLayer(california, {}, 'California Boundary')
Map.addLayerControl()
Map


Processing: DJF (Dec-Feb)
Processing: MAM (Mar-May)
Processing: JJA (Jun-Aug)
Processing: SON (Sep-Nov)


Map(center=[37.5, -119], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

export EVM and BIM to earth engine assets ubder a folder.

MTBS

In [ ]:
!gsutil ls gs://california-fire-model/evms/*/*.tif



CommandException: One or more URLs matched no objects.


In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='california-fire-model')

california = ee.FeatureCollection("TIGER/2018/States") \
              .filter(ee.Filter.eq("NAME", "California")).geometry()

image = ee.Image.constant(1).clip(california)

task = ee.batch.Export.image.toCloudStorage(
    image=image,
    description='test_export',
    bucket='california-fire-model',
    fileNamePrefix='test/test_image',
    region=california.bounds(),
    scale=500,
    crs='EPSG:4326',
    maxPixels=1e13
)
task.start()
print("🚀 Export triggered!")


🚀 Export triggered!


In [ ]:
!gsutil ls gs://california-fire-model/test/


CommandException: One or more URLs matched no objects.


In [ ]:
# Step 1: Install and import packages
!pip install earthengine-api geemap --quiet

import ee
import geemap

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='california-fire-model')

# Step 2: Define California geometry
california = ee.FeatureCollection("TIGER/2018/States") \
              .filter(ee.Filter.eq("NAME", "California")) \
              .geometry()

# Step 3: Define seasons
seasons = {
    'DJF': ([12, 1, 2], 'Dec-Feb'),
    'MAM': ([3, 4, 5], 'Mar-May'),
    'JJA': ([6, 7, 8], 'Jun-Aug'),
    'SON': ([9, 10, 11], 'Sep-Nov')
}

def get_season_dates(year, months):
    start_month = months[0]
    end_month = months[-1]
    if start_month > end_month:
        start = ee.Date(f'{year-1}-{start_month:02d}-01')
        end = ee.Date(f'{year}-{end_month:02d}-28')
    else:
        start = ee.Date(f'{year}-{start_month:02d}-01')
        end = ee.Date(f'{year}-{end_month:02d}-28')
    return start, end

# Function to upscale to 500m resolution
def upscale(img):
    return img.resample('bilinear').reproject(crs='EPSG:4326', scale=500)

# Step 4: Generate EVMs and Export to GCS
bucket_name = 'california-fire-model'
scale = 500
crs = 'EPSG:4326'
evms = {}

for season_key, (months, label) in seasons.items():
    print(f"Processing: {season_key} ({label})")

    start, end = get_season_dates(2005, months)

    # === PRISM ===
    prism = ee.ImageCollection('OREGONSTATE/PRISM/AN81m').filterDate(start, end)
    tmax = upscale(prism.select('tmax').mean().rename('tmax'))
    tmin = upscale(prism.select('tmin').mean().rename('tmin'))
    precip = upscale(prism.select('ppt').sum().rename('precip'))

    # === GRIDMET ===
    gridmet = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET').filterDate(start, end)
    wind = upscale(gridmet.select('vs').mean().rename('wind'))
    humid = upscale(gridmet.select('sph').mean().rename('humidity'))
    eto = upscale(gridmet.select('eto').sum().rename('eto'))
    aridity = upscale(gridmet.select('pr').sum().divide(eto).rename('aridity'))

    # === MODIS ===
    ndvi = upscale(ee.ImageCollection('MODIS/006/MOD13A1')
                   .filterDate(start, end).select('NDVI').mean().rename('NDVI'))
    lai = upscale(ee.ImageCollection('MODIS/006/MCD15A3H')
                  .filterDate(start, end).select('Lai').mean().rename('LAI'))

    # === Combine all and cast to float ===
    evm = ee.Image.cat([
        tmax, tmin, precip, wind, humid, eto,
        aridity, ndvi, lai
    ]).clip(california).toFloat()  # ✅ Ensures all bands are Float32

    evms[season_key] = evm

    # === Export to Cloud Storage ===
    file_prefix = f'evms/{season_key}/EVM_{season_key}_2005'
    task = ee.batch.Export.image.toCloudStorage(
        image=evm,
        description=f'Export_{season_key}_2005',
        bucket=bucket_name,
        fileNamePrefix=file_prefix,
        region=california,
        scale=scale,
        crs=crs,
        maxPixels=1e13
    )
    task.start()
    print(f'🚀 Export started → gs://{bucket_name}/{file_prefix}.tif')

# Step 5: Display PRISM tmax layer
Map = geemap.Map(center=[37.5, -119], zoom=6)
season_vis_ranges = {
    'DJF': (0, 20),
    'MAM': (5, 30),
    'JJA': (15, 45),
    'SON': (5, 30),
}
for season_key in seasons.keys():
    vmin, vmax = season_vis_ranges[season_key]
    Map.addLayer(evms[season_key], {
        'bands': ['tmax'],
        'min': vmin,
        'max': vmax,
        'palette': ['blue', 'green', 'yellow', 'red']
    }, f'{season_key} PRISM Tmax (°C)')
Map.addLayer(california, {}, 'California Boundary')
Map.addLayerControl()
Map


Processing: DJF (Dec-Feb)
🚀 Export started → gs://california-fire-model/evms/DJF/EVM_DJF_2005.tif
Processing: MAM (Mar-May)
🚀 Export started → gs://california-fire-model/evms/MAM/EVM_MAM_2005.tif
Processing: JJA (Jun-Aug)
🚀 Export started → gs://california-fire-model/evms/JJA/EVM_JJA_2005.tif
Processing: SON (Sep-Nov)
🚀 Export started → gs://california-fire-model/evms/SON/EVM_SON_2005.tif


Map(center=[37.5, -119], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

In [ ]:
!gsutil ls gs://california-fire-model/evms/



gs://california-fire-model/evms/DJF/
gs://california-fire-model/evms/JJA/
gs://california-fire-model/evms/SON/


export all EVMS from 2000 to 2025.

In [ ]:
# Step 1: Install and import packages
!pip install earthengine-api geemap --quiet

import ee
import geemap

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='california-fire-model')

# Step 2: Define California geometry
california = ee.FeatureCollection("TIGER/2018/States") \
              .filter(ee.Filter.eq("NAME", "California")) \
              .geometry()

# Step 3: Define seasons
seasons = {
    'DJF': ([12, 1, 2], 'Dec-Feb'),
    'MAM': ([3, 4, 5], 'Mar-May'),
    'JJA': ([6, 7, 8], 'Jun-Aug'),
    'SON': ([9, 10, 11], 'Sep-Nov')
}

def get_season_dates(year, months):
    start_month = months[0]
    end_month = months[-1]
    if start_month > end_month:
        start = ee.Date(f'{year-1}-{start_month:02d}-01')
        end = ee.Date(f'{year}-{end_month:02d}-28')
    else:
        start = ee.Date(f'{year}-{start_month:02d}-01')
        end = ee.Date(f'{year}-{end_month:02d}-28')
    return start, end

# Function to upscale to 500m resolution and cast to Float32
def upscale(img):
    return img.resample('bilinear').reproject(crs='EPSG:4326', scale=500).toFloat()

# Step 4: Generate and Export EVMs
bucket_name = 'california-fire-model'
scale = 500
crs = 'EPSG:4326'

for year in range(2000, 2026):
    for season_key, (months, label) in seasons.items():
        print(f"📦 Exporting EVM: {season_key}, {year}")
        start, end = get_season_dates(year, months)

        try:
            # PRISM
            prism = ee.ImageCollection('OREGONSTATE/PRISM/AN81m').filterDate(start, end)
            tmax = upscale(prism.select('tmax').mean().rename('tmax'))
            tmin = upscale(prism.select('tmin').mean().rename('tmin'))
            precip = upscale(prism.select('ppt').sum().rename('precip'))

            # GRIDMET
            gridmet = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET').filterDate(start, end)
            wind = upscale(gridmet.select('vs').mean().rename('wind'))
            humid = upscale(gridmet.select('sph').mean().rename('humidity'))
            eto = upscale(gridmet.select('eto').sum().rename('eto'))
            aridity = upscale(gridmet.select('pr').sum().divide(eto).rename('aridity'))

            # MODIS
            ndvi = upscale(ee.ImageCollection('MODIS/006/MOD13A1')
                           .filterDate(start, end).select('NDVI').mean().rename('NDVI'))
            lai = upscale(ee.ImageCollection('MODIS/006/MCD15A3H')
                          .filterDate(start, end).select('Lai').mean().rename('LAI'))

            # Combine all bands
            evm = ee.Image.cat([
                tmax, tmin, precip, wind, humid, eto,
                aridity, ndvi, lai
            ]).clip(california).toFloat()

            # Export
            file_prefix = f'evms/{season_key}/EVM_{season_key}_{year}'
            task = ee.batch.Export.image.toCloudStorage(
                image=evm,
                description=f'Export_{season_key}_{year}',
                bucket=bucket_name,
                fileNamePrefix=file_prefix,
                region=california,
                scale=scale,
                crs=crs,
                maxPixels=1e13
            )
            task.start()
        except Exception as e:
            print(f"❌ Failed export {season_key}_{year}: {e}")


📦 Exporting EVM: DJF, 2000
📦 Exporting EVM: MAM, 2000
📦 Exporting EVM: JJA, 2000
📦 Exporting EVM: SON, 2000
📦 Exporting EVM: DJF, 2001
📦 Exporting EVM: MAM, 2001
📦 Exporting EVM: JJA, 2001
📦 Exporting EVM: SON, 2001
📦 Exporting EVM: DJF, 2002
📦 Exporting EVM: MAM, 2002
📦 Exporting EVM: JJA, 2002
📦 Exporting EVM: SON, 2002
📦 Exporting EVM: DJF, 2003
📦 Exporting EVM: MAM, 2003
📦 Exporting EVM: JJA, 2003
📦 Exporting EVM: SON, 2003
📦 Exporting EVM: DJF, 2004
📦 Exporting EVM: MAM, 2004
📦 Exporting EVM: JJA, 2004
📦 Exporting EVM: SON, 2004
📦 Exporting EVM: DJF, 2005
📦 Exporting EVM: MAM, 2005
📦 Exporting EVM: JJA, 2005
📦 Exporting EVM: SON, 2005
📦 Exporting EVM: DJF, 2006
📦 Exporting EVM: MAM, 2006
📦 Exporting EVM: JJA, 2006
📦 Exporting EVM: SON, 2006
📦 Exporting EVM: DJF, 2007
📦 Exporting EVM: MAM, 2007
📦 Exporting EVM: JJA, 2007
📦 Exporting EVM: SON, 2007
📦 Exporting EVM: DJF, 2008
📦 Exporting EVM: MAM, 2008
📦 Exporting EVM: JJA, 2008
📦 Exporting EVM: SON, 2008
📦 Exporting EVM: DJF, 2009
📦

In [ ]:
!gsutil ls gs://california-fire-model/evms/


gs://california-fire-model/evms/DJF/
gs://california-fire-model/evms/JJA/
gs://california-fire-model/evms/MAM/
gs://california-fire-model/evms/SON/


In [ ]:
import ee
ee.Initialize(project='california-fire-model')

tasks = ee.batch.Task.list()
for task in tasks:
    print(f"{task.config['description']}: {task.state}")


Export_SON_2025: READY
Export_JJA_2025: READY
Export_MAM_2025: READY
Export_DJF_2025: READY
Export_SON_2024: READY
Export_JJA_2024: READY
Export_MAM_2024: READY
Export_DJF_2024: READY
Export_SON_2023: READY
Export_JJA_2023: READY
Export_MAM_2023: READY
Export_DJF_2023: READY
Export_SON_2022: READY
Export_JJA_2022: READY
Export_MAM_2022: READY
Export_DJF_2022: READY
Export_SON_2021: READY
Export_JJA_2021: READY
Export_MAM_2021: READY
Export_DJF_2021: READY
Export_SON_2020: READY
Export_JJA_2020: READY
Export_MAM_2020: READY
Export_DJF_2020: READY
Export_SON_2019: READY
Export_JJA_2019: READY
Export_MAM_2019: READY
Export_DJF_2019: READY
Export_SON_2018: READY
Export_JJA_2018: READY
Export_MAM_2018: READY
Export_DJF_2018: READY
Export_SON_2017: READY
Export_JJA_2017: READY
Export_MAM_2017: READY
Export_DJF_2017: READY
Export_SON_2016: READY
Export_JJA_2016: READY
Export_MAM_2016: READY
Export_DJF_2016: READY
Export_SON_2015: READY
Export_JJA_2015: READY
Export_MAM_2015: READY
Export_DJF_

export these ignition risk maps/ images in google bucket.

In [ ]:
import ee
import geemap

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='california-fire-model')

# Define California geometry
california = ee.FeatureCollection("TIGER/2018/States") \
    .filter(ee.Filter.eq("NAME", "California")) \
    .geometry()

# MODIS Burned Area
modis_ba = ee.ImageCollection("MODIS/061/MCD64A1").select('BurnDate')

# Seasons
seasons = {
    'DJF': ([12, 1, 2], 'Dec-Feb'),
    'MAM': ([3, 4, 5], 'Mar-May'),
    'JJA': ([6, 7, 8], 'Jun-Aug'),
    'SON': ([9, 10, 11], 'Sep-Nov')
}

def get_season_dates(year, months):
    start_month = months[0]
    end_month = months[-1]
    if start_month > end_month:
        return ee.Date(f'{year-1}-{start_month:02d}-01'), ee.Date(f'{year}-{end_month:02d}-28')
    else:
        return ee.Date(f'{year}-{start_month:02d}-01'), ee.Date(f'{year}-{end_month:02d}-28')

# Set GCS bucket
bucket_name = 'california-fire-model'
scale = 500
crs = 'EPSG:4326'

for year in range(2000, 2026):
    for season_key, (months, label) in seasons.items():
        print(f"📦 Exporting BIM for: {season_key} {year}")
        start, end = get_season_dates(year, months)

        # Create binary ignition map for the season and year
        seasonal_fire = modis_ba.filterDate(start, end).map(lambda img: img.gt(0).rename('Fire'))
        count = seasonal_fire.size()

        season_sum = ee.Image(ee.Algorithms.If(
            count.gt(0),
            seasonal_fire.sum().clip(california),
            ee.Image.constant(0).rename('Fire').clip(california)
        ))

        bim = season_sum.gt(0).rename('BIM').toFloat()

        # Export to GCS
        file_prefix = f'bims/{season_key}/BIM_{season_key}_{year}'
        task = ee.batch.Export.image.toCloudStorage(
            image=bim,
            description=f'Export_BIM_{season_key}_{year}',
            bucket=bucket_name,
            fileNamePrefix=file_prefix,
            region=california,
            scale=scale,
            crs=crs,
            maxPixels=1e13
        )
        task.start()
        print(f"🚀 Export started → gs://{bucket_name}/{file_prefix}.tif")


📦 Exporting BIM for: DJF 2000
🚀 Export started → gs://california-fire-model/bims/DJF/BIM_DJF_2000.tif
📦 Exporting BIM for: MAM 2000
🚀 Export started → gs://california-fire-model/bims/MAM/BIM_MAM_2000.tif
📦 Exporting BIM for: JJA 2000
🚀 Export started → gs://california-fire-model/bims/JJA/BIM_JJA_2000.tif
📦 Exporting BIM for: SON 2000
🚀 Export started → gs://california-fire-model/bims/SON/BIM_SON_2000.tif
📦 Exporting BIM for: DJF 2001
🚀 Export started → gs://california-fire-model/bims/DJF/BIM_DJF_2001.tif
📦 Exporting BIM for: MAM 2001
🚀 Export started → gs://california-fire-model/bims/MAM/BIM_MAM_2001.tif
📦 Exporting BIM for: JJA 2001
🚀 Export started → gs://california-fire-model/bims/JJA/BIM_JJA_2001.tif
📦 Exporting BIM for: SON 2001
🚀 Export started → gs://california-fire-model/bims/SON/BIM_SON_2001.tif
📦 Exporting BIM for: DJF 2002
🚀 Export started → gs://california-fire-model/bims/DJF/BIM_DJF_2002.tif
📦 Exporting BIM for: MAM 2002
🚀 Export started → gs://california-fire-model/bims/M

 1: Setup and Mount Google Cloud
We’ll begin by:

Authenticating to GCP

Verifying access to your bucket (california-fire-model)

Installing necessary packages



In [ ]:
  # PART 1: Authenticate and Set Up Environment
  from google.colab import auth
  auth.authenticate_user()

  # Login to access Google Cloud
  !gcloud auth application-default login --quiet

  # Your GCS bucket name
  gcs_bucket = 'california-fire-model'
  gcs_path = f'gs://{gcs_bucket}'

  # Install required libraries
  !pip install rasterio geopandas tensorflow tensorflow_io --quiet


Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=QoWZKuDIN9cj90ePE1onm3vLJc1Ihg&prompt=consent&token_usage=remote&access_type=offline&code_challenge=kpJsaRooCSXAGktPO6uYSMYxTTnDSzlj04pK79aoLo8&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AVMBsJjBpGQ0iVOeZYE6KwmZ6yYHlI2c9xHOfF1roXAA2DnZLTDn52bX0n3FVyYp4OyBtg

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [ ]:
!pip install google-cloud-storage rasterio geopandas tensorflow tensorflow-io --quiet


In [ ]:
# Authenticate Colab session
from google.colab import auth
auth.authenticate_user()

# Set GCS config
gcs_bucket = 'california-fire-model'
gcs_path = f'gs://{gcs_bucket}'

# Login to access Google Cloud
!gcloud auth application-default login --quiet

# Install required libraries
!pip install rasterio geopandas tensorflow tensorflow_io gcsfs --quiet


/usr/local/lib/python3.11/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=MwfljCxuwhysQGzNGpvvQwVGwiWvjG&prompt=consent&token_usage=remote&access_type=offline&code_challenge=14fTwGLphxvIFWZcK2pTFu11o4_fhmu3JJz3yxfUvgY&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AVMBsJjy25M8BEPcC5dVtrVLNpwso9N88_mMxB7VA5g-HWkpeGLBBJHsqLBthhacgpn-ew

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

 Patch Extraction Logic
This will:

Iterate over years 2000–2016 and seasons DJF, MAM, JJA, SON

Match EVM and BIM files

Extract valid 32×32 patches centered where BIM pixel is 0, 1, or 2

Save patches and labels to .npy format

In [ ]:
# Colab-compatible memory-efficient patch extraction

import os
import numpy as np
import rasterio
from rasterio.windows import Window
from google.cloud import storage
from tqdm import tqdm

# Define parameters
gcs_bucket = 'california-fire-model'
gcs_path = f'gs://{gcs_bucket}'
local_evm_dir = '/content/evms'
local_bim_dir = '/content/bims'
output_patch_dir = '/content/patches'

os.makedirs(local_evm_dir, exist_ok=True)
os.makedirs(local_bim_dir, exist_ok=True)
os.makedirs(output_patch_dir, exist_ok=True)

seasons = ['DJF', 'MAM', 'JJA', 'SON']
years = list(range(2000, 2017))  # Only training years
patch_size = 32

# Initialize GCS client
client = storage.Client(project="california-fire-model")


# Helper: Download file from GCS
def download_blob(gcs_uri, local_path):
    bucket_name = gcs_uri.split('/')[2]
    blob_path = '/'.join(gcs_uri.split('/')[3:])
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_path)
    blob.download_to_filename(local_path)

# Helper: Extract patches

def extract_and_save_patches(evm_path, bim_path, season, year):
    with rasterio.open(evm_path) as evm_src, rasterio.open(bim_path) as bim_src:
        height, width = bim_src.shape
        evm_data = evm_src.read()  # shape: (bands, height, width)
        bim_data = bim_src.read(1)  # shape: (height, width)

        X_patches = []
        y_patches = []

        for i in range(0, height - patch_size, patch_size):
            for j in range(0, width - patch_size, patch_size):
                bim_patch = bim_data[i:i+patch_size, j:j+patch_size]
                if np.any(np.isin(bim_patch, [0, 1, 2])):
                    evm_patch = evm_data[:, i:i+patch_size, j:j+patch_size]  # shape: (bands, 32, 32)
                    X_patches.append(evm_patch.astype(np.float32))
                    y_patches.append(bim_patch.astype(np.uint8))

        X_patches = np.array(X_patches)
        y_patches = np.array(y_patches)

        # Save to .npy
        np.save(os.path.join(output_patch_dir, f'X_{season}_{year}.npy'), X_patches)
        np.save(os.path.join(output_patch_dir, f'y_{season}_{year}.npy'), y_patches)
        print(f"✅ Saved {len(X_patches)} patches for {season} {year}")

# Main loop
for season in seasons:
    for year in tqdm(years, desc=f"Processing {season}"):
        evm_gcs = f'{gcs_path}/evms/{season}/EVM_{season}_{year}.tif'
        bim_gcs = f'{gcs_path}/bims/{season}/BIM_{season}_{year}.tif'

        local_evm = os.path.join(local_evm_dir, f'EVM_{season}_{year}.tif')
        local_bim = os.path.join(local_bim_dir, f'BIM_{season}_{year}.tif')

        try:
            download_blob(evm_gcs, local_evm)
            download_blob(bim_gcs, local_bim)
            extract_and_save_patches(local_evm, local_bim, season, year)
        except Exception as e:
            print(f"❌ Error processing {season} {year}: {e}")


/usr/local/lib/python3.11/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
Processing DJF:   0%|          | 0/17 [00:00<?, ?it/s]/tmp/ipython-input-2-4171324147.py:54: RuntimeWarning: invalid value encountered in cast
  y_patches.append(bim_patch.astype(np.uint8))
Processing DJF:   6%|▌         | 1/17 [00:04<01:07,  4.19s/it]

✅ Saved 2246 patches for DJF 2000


Processing DJF:  12%|█▏        | 2/17 [00:07<00:50,  3.40s/it]

✅ Saved 14 patches for DJF 2001


Processing DJF:  18%|█▊        | 3/17 [00:10<00:44,  3.20s/it]

...

✅ Saved 51 patches for SON 2014


Processing SON:  94%|█████████▍| 16/17 [00:46<00:03,  3.03s/it]

✅ Saved 68 patches for SON 2015


Processing SON: 100%|██████████| 17/17 [00:48<00:00,  2.87s/it]

✅ Saved 48 patches for SON 2016


In [ ]:
from google.colab import files
uploaded = files.upload()


In [3]:
import nbformat

# Replace with your actual notebook file
notebook_path = "wildfire_model.ipynb"

with open(notebook_path, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=nbformat.NO_CONVERT)

# Remove the faulty metadata
if "widgets" in nb.metadata:
    del nb.metadata["widgets"]

# Also clean per-cell widget metadata
for cell in nb.cells:
    if "widgets" in cell.get("metadata", {}):
        del cell["metadata"]["widgets"]

# Save the cleaned notebook
with open("cleaned_notebook.ipynb", "w", encoding="utf-8") as f:
    nbformat.write(nb, f)

print("Notebook cleaned and saved as cleaned_notebook.ipynb")


FileNotFoundError: [Errno 2] No such file or directory: 'wildfire_model.ipynb'